In [ ]:
import scipy.misc
from scipy import ndimage
import sys
import os
import glob
import json
from collections import defaultdict
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
from PIL import Image
from keras.utils import np_utils
from keras.models import Model, model_from_json
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score, confusion_matrix, roc_curve, auc

%matplotlib inline

In [ ]:
def preprocess_input(x0):
    return ((x0/255.)-0.5)*2.

In [ ]:
def data_setting(condition_1, condition_2):
    
    global X_test, Y_test, y_pred
    
    test_X = np.concatenate((condition_1, condition_2),axis=0)
    test_y = np.concatenate((np.tile(np.array([[0]]),(condition_1.shape[0],1)),
                             np.tile(np.array([[1]]),(condition_2.shape[0],1))
                            ),axis=0)
    X_test = preprocess_input(np.copy(test_X))
    y_test = np.copy(test_y)

    X_test = X_test.reshape(X_test.shape[0], scale_v*2, scale_h*2, n_chan)
    Y_test = np_utils.to_categorical(y_test, 2)
    
    Y_pred = model.predict(X_test, batch_size=batch_size)
    y_pred = np.argmax(Y_pred, axis=1)

    return X_test, Y_test, Y_pred, y_pred

In [ ]:
def save_parameters(network_name, condition_1, condition_2):
   
    global F1_score, Accuracy, Precision, Recall, confusion_m
    
    data_setting(condition_1, condition_2)
    
    #Final parameters
    F1_score = f1_score(np.argmax(Y_test,1), y_pred, average='binary')
    Accuracy = accuracy_score(np.argmax(Y_test,1), y_pred)
    Precision = precision_score(np.argmax(Y_test,1), y_pred, average='binary')
    Recall = recall_score(np.argmax(Y_test,1), y_pred, average='binary')
    
    parameter_dict = {network_name:[F1_score, Accuracy, Precision, Recall]}
    parameter_dataframe = pd.DataFrame(parameter_dict,index=["F1_score", "Accuracy", "Precision", "Recall"])
    
    print(parameter_dataframe)
    parameter_dataframe.to_csv(network_name + "_final_parameter" + ".csv")
    
    #Confusion matrix
    confusion_m = confusion_matrix(np.argmax(Y_test,1), y_pred)
    confusion_dataframe = pd.DataFrame(confusion_m, index=["Answer:0", "Answer:1"])
    confusion_dataframe.columns = ["Prediction:0", "Prediction:1"]
    print(confusion_dataframe)
    confusion_dataframe.to_csv(network_name + "_confusion_matrix" + ".csv")
    
    #AUC of ROC curve

    prob = model.predict_proba(X_test)[:,1]
    fpr, tpr, threshold = roc_curve(np.argmax(Y_test,1), prob)
    roc_auc = auc(fpr, tpr)
    print("AUC: {0}".format(roc_auc))
    roc_data = [fpr, tpr]
    roc_dataframe = pd.DataFrame(roc_data, index=["False positive rate", "True positive rate"])
    print(roc_dataframe)
    roc_dataframe.to_csv(network_name + "_roc_auc" + ".csv")
      
    plt.plot(fpr, tpr, color='red',lw= 2, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='black', lw= 2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="best")
     
    plt.savefig(network_name + "_roc_auc" + ".jpg")
    plt.show()

In [ ]:
# Load trained CNN data
json_file_path = '/home/Demo/save_data/training_save_data/test.json'
h5_file_path = '/home/Demo/save_data/training_save_data/test.h5'

In [ ]:
# Prameters for validation

n_chan=3            # Number of channels of images

scale_v = 25        # Image width/2 (px)
scale_h = 25       # Image height/2 (px)

batch_size = 128

In [ ]:
# load npy
control_npy = np.load("/home/Demo/npy/validation_npy/test.npy")   #Load numpy file of healthy condition
stress_npy = np.load("/home/Demo/npy/validation_npy/test_st.npy")  #Load numpy file of stress condition

In [ ]:
print(control_npy.shape)
print(stress_npy.shape)

In [ ]:
model = model_from_json(open(json_file_path).read())
model.load_weights(h5_file_path)

In [ ]:
network_name = "/home/Demo/save_data/validation_save_data/validation_test"  #Directory path and name to save results
save_parameters(network_name, control_npy, stress_npy)